# Function Calling with Chat API

본 튜토리얼을 통해 Friendli Serverless Endpoints의 Chat 모델을 활용하여 동적으로 함수 호출에 필요한 인자 값을 생성하고, 이를 활용하여 함수를 호출하여 정보를 얻는 방법에 대해서 알을봅시다.

Function calling이란 모델에게 질문과 더불어 사전에 정의한 함수의 정보(e.g., 함수 이름, 함수의 인자들, 함수의 설명)를 함께 제공했을 때, 그 질문에 알맞은 함수 호출을 할 수 있도록 모델이 함수의 인자 값을 생성해 주는 기능을 말합니다. Function calling 기능은 다음과 같은 단계를 거쳐 활용이 가능합니다.

1. 함수의 인자 값을 생성
2. 생성된 함수 인자 값을 활용하여 함수 호출
3. 함수 호출 결과를 담아서 최종 답변 생성

모델을 통해서 질문에 맞는 함수 인자 값을 생성하고, 함수 호출로 얻은 정보를 모델의 답변 생성에 활용한다는 점에서 function calling 기능은 Retrieval Augmented Generation(RAG)의 일종이라고 볼 수 있습니다.

본 튜토리얼에서는 사용자가 "지금 날씨에 어떤 옷을 입어야 해?"와 같은 질문을 했을 때 기상청 등에서 실시간으로 받아온 정보에 기반하여 옷차림을 추천해 주는 간단한 예제를 다루어 보겠습니다.

## Installing Client

Friendli Serverless Endpoints는 OpenAI와 동일한 스키마의 Chat API를 제공합니다. Function calling 기능 역시 OpenAI Chat API와 동일한 옵션으로 사용이 가능합니다.
튜토리얼 진행을 위해 OpenAI Python 클라이언트를 설치합니다. 

In [ ]:
!pip install openai

## Defining Function

호출하고 싶은 함수를 정의합니다. 이 예제에서는 미리 정의된 입력(=지역)에 대해 정해진 결과(=기온)를 반환하는 dummy 함수를 정의하였습니다. 물론 실제 응용 사례에서는 날씨 서버에서 정보를 가져오는 함수를 정의해서 사용을 해야겠죠?

In [1]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "seoul" in location.lower():
        return 10
    elif "san francisco" in location.lower():
        return 72
    elif "paris" in location.lower():
        return 22
    else:
        return 20

## Sending Function Calling Request

이제 Chat API를 통해 함수 호출에 사용될 인자를 생성해 봅시다.

우선 Friendli Serverless Endpoints에 API call 시 사용할 **Personal Access Token**을 준비합니다.

In [2]:
import getpass

friendli_token = getpass.getpass()

사용자가 오늘과 같은 날씨에 어떤 옷을 입어야 하는지 질문을 한다고 해봅시다.

In [3]:
question_from_user = "I live in Seoul. What should I wear for today's weather?"

이제 OpenAI client의 `base_url`을 Friendli Serverless Endpoints의 주소인 `https://inference.friendli.ai/v1`으로 변경하여 요청을 보내보겠습니다.

- 이때 사용자의 질문은 `messages` 옵션에 `user` role의 `content`로 들어갑니다.
- 위에서 정의했던 `get_current_weather` 함수의 정보는 `tools` 옵션에 들어갑니다. 이때 `parameters`에는 위에서 정의했던 함수의 인자 정보가 [JSON Schema](https://json-schema.org/) 형식으로 포함되어야 합니다.

In [4]:
from openai import OpenAI

client = OpenAI(base_url="https://inference.friendli.ai/v1", api_key=friendli_token)

messages = [
    {
        "role": "user",
        "content": question_from_user,
    },
]

tools=[
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # 위에서 정의한 함수의 이름
            "description": "Get the current weather information in a given location.",  # 함수에 대한 설명
            "parameters": {  # JSON Schema로 정의한 함수의 인자 정보
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of current location e.g., Seoul",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature degree.",
                    },
                },
            },
        },
    },
]

chat = client.chat.completions.create(
    model="mistral-7b-instruct-v0-3",
    messages=messages,
    tools=tools,
    temperature=0,
    frequency_penalty=1,
)
print(chat)

ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_020', function=Function(arguments='{"location":"Seoul","unit":"celsius"}', name='get_current_weather'), type='function')]))], created=1716527581, model=None, object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=118, total_tokens=144))


생성된 답변의 `arguments` 필드에 포함된 함수의 인자 값을 위에서 정의된 함수에 그대로 unpacking 하여 호출하면 사용자의 질문의 의도가 반영된 함수 호출 결과를 생성할 수 있습니다.

In [5]:
import json

func_kwargs = json.loads(chat.choices[0].message.tool_calls[0].function.arguments)

weather_info = get_current_weather(**func_kwargs)
print(weather_info)

10


함수 호출을 통해서 받아온 날씨 정보를 바탕으로 "I live in Seoul. What should I wear for today's weather?"에 대한 최종적인 답변을 생성 해봅시다.
이전 Chat API call에서 받아온 함수 인자 정보를 "assistant" 메세지로 추가하고, `get_current_weather` 함수를 호출해서 얻은 결과를 "tool" 메세지로 추가하여 다시 Chat API를 호출합니다. 

In [6]:
messages.append(
    {
        "role": "assistant",
        "tool_calls": [
            tool_call.model_dump()
            for tool_call in chat.choices[0].message.tool_calls
        ]
    }
)

messages.append(
    {
        "role": "tool",
        "content": str(weather_info),
        "tool_call_id": chat.choices[0].message.tool_calls[0].id
    }
)

chat_w_info = client.chat.completions.create(
    model="mistral-7b-instruct-v0-3",
    tools=tools,
    messages=messages,
)
print(chat_w_info.choices[0].message.content)

Today in Seoul, it's 10 degrees Celsius. I would recommend dressing in layers for comfort, with a jacket or sweater for cooler moments during the day.


## Caveat

### Friendli Serverless Endpoints

- 현재 Friendli Serverless Endpoints에서 function calling이 지원되는 모델은 `mistral-7b-instruct-v0-3`과 `mixtral-8x7b-instruct-v0-1`입니다.

### Friendli Dedicated Endpoints

- Function calling 기능을 고려한 chat template을 사용해야 문제가 없습니다. 즉, role이 `tool`인 message, 그리고 `tools` 옵션의 정보들을 렌더링 가능하도록 chat template을 작성해야 합니다. Friendli Dedicated Endpoints에 원하는 모델을 배포하여 사용할 경우 `tokenizer_config.json`의 "chat_template" 필드에 function calling을 고려한 chat template이 들어가야 모델이 적절한 답변을 생성할 수 있습니다. Chat template 작성 방법에 대한 가이드는 [Templates for Chat Models](https://huggingface.co/docs/transformers/chat_templating)를 참고 바랍니다.
- Function calling 기능을 고려하여 학습된 모델을 사용해야 function calling이 제대로 동작합니다 (e.g., `mistralai/Mistral-7B-Instruct-v0.3`).

### Common: Invalid JSON

생성 언어 모델의 잘 알려진 문제 중 하나로 같은 단어나 문자를 무한히 반복해서 생성하는 문제가 있습니다. 이 문제는 function calling 기능을 사용할 때 특히 치명적일 수 있습니다. 출력을 무한히 길게 생성하다 보면 정상적인 JSON 객체로 변환이 불가능한 텍스트가 생성될 수 있기 때문입니다. 예를 들어, 모델이 `\n`를 무한히 반복하는 경우 `{`로 시작해서 `}`로 끝나는 유효한 형식이 아니라, `{\n\n ... \n`와 같은 잘못된 형식의 결과가 생성됩니다. 따라서 function calling의 응답 결과가 제대로 된 JSON 형식으로 되어 있는지 항상 확인이 필요합니다.

다음 두 가지 경우엔 모델이 유효한 JSON 객체를 생성하지 못한 상황에 해당합니다:

1. **`tool_calls` 필드가 응답에 포함되지 않음**: 이 경우 일반적인 chat API와 같이 `tool_calls` 대신 `content` 필드에 응답 텍스트가 포함됩니다.
2. **`finish_reason="length"`인 경우**: token 길이의 제약으로 인해 완전한 결과를 생성하지 못한 경우엔 유효한 JSON 형식이 될 수가 없습니다.

> #### Tips
> - 추론 요청에 `max_tokens` 필드를 포함하는 것을 강하게 권장합니다. 해당 필드가 없는 경우 추론 엔진은 기본적으로 응답 길이에 제한을 두지 않으며, 따라서 동어 반복(예: `\n` 가 무한히 반복됨)이 발생한 경우 응답이 오기까지 길게는 수십분이 소요될 수 있습니다.
> - 동어 반복을 방지하기 위해 `frequency_penalty` 또는 `presence_penalty` 옵션을 사용하는 것도 좋은 방법입니다.